In [ ]:
# ==========================================================
# GPU Yield Model: Aggregation + Log Yields + Spatial CV + Early Stopping
# ==========================================================
import os
import pandas as pd
import numpy as np
from cuml.ensemble import RandomForestRegressor as cuRF
from sklearn.model_selection import GroupKFold, ParameterGrid
from sklearn.metrics import r2_score, mean_squared_error
from tqdm.notebook import tqdm
import joblib
from time import perf_counter


# ----------------------------------------------------------
# 1. File paths and output setup
# ----------------------------------------------------------
sat_path = "/content/drive/MyDrive/Research/Crop Yield Prediction/EE_harvest_ml.csv"
yield_path = "/content/drive/MyDrive/Research/Crop Yield Prediction/LSMS Data/Plot_dataset.dta"
model_dir = "/content/drive/MyDrive/Research/Crop Yield Prediction/models_gpu_spatialcv"
os.makedirs(model_dir, exist_ok=True)

# ----------------------------------------------------------
# 2. Load and preprocess data
# ----------------------------------------------------------
print("🔹 Loading datasets...")
yield_df = pd.read_stata(yield_path)
sat_df = pd.read_csv(sat_path)

# Filter out implausible yields
yield_df = yield_df[(yield_df['yield_kg'] > 0) & (yield_df['yield_kg'] < 15000)]
yield_df = yield_df.dropna(subset=['yield_kg', 'lat_modified', 'lon_modified'])

# Aggregate to (country, wave, season, lat, lon)
agg_yields = (
    yield_df.groupby(['country', 'wave', 'season', 'lat_modified', 'lon_modified'])
    .agg(mean_yield=('yield_kg', 'mean'))
    .reset_index()
)

# Log-transform target
agg_yields['log_yield'] = np.log1p(agg_yields['mean_yield'])

# Merge with satellite data
df = pd.merge(
    agg_yields,
    sat_df,
    on=['country', 'wave', 'season', 'lat_modified', 'lon_modified'],
    how='inner'
)

# Drop missing features
df = df.dropna()
print(f"✅ Final merged shape: {df.shape}")

# ----------------------------------------------------------
# 3. Define variables
# ----------------------------------------------------------
merge_vars = ['country','wave','season','lat_modified','lon_modified']
y_var = 'log_yield'
x_vars = [c for c in df.columns if c not in merge_vars + ['mean_yield', 'log_yield']]

X = df[x_vars]
y = df[y_var]

# ----------------------------------------------------------
# 4. GroupKFold setup for spatial CV
# ----------------------------------------------------------
groups = (
    df['country'].astype(str) + "_" +
    df['wave'].astype(str) + "_" +
    df['lat_modified'].astype(str) + "_" +
    df['lon_modified'].astype(str)
)
cv = GroupKFold(n_splits=5)

# ----------------------------------------------------------
# 5. Parameter grid (moderate size, safe for GPU)
# ----------------------------------------------------------
param_grid = {
    'n_estimators': [300, 600, 900],
    'max_depth': [12, 20, 32, 48],
    'max_features': [0.5, 0.8],
    'min_samples_split': [2, 8],
    'min_samples_leaf': [1, 4],
    'bootstrap': [True],
}
grid_list = list(ParameterGrid(param_grid))
rng = np.random.default_rng(42)
grid_list = [grid_list[i] for i in rng.permutation(len(grid_list))]
print(f"Total parameter combos (shuffled): {len(grid_list)}")

# ----------------------------------------------------------
# 6. Helper functions for tuning and training
# ----------------------------------------------------------
def tune_with_spatial_cv_cuml(
    X, y, groups, grid_list,
    patience=15, min_delta=1e-3, desc_label="Tuning"
):
    best_r2 = -np.inf
    best_params = None
    no_improve = 0
    start = perf_counter()

    for p in tqdm(grid_list, desc=desc_label, ncols=100):
        r2_fold = []
        rmse_fold = []
        for tr_idx, va_idx in cv.split(X, y, groups):
            rf = cuRF(
                n_estimators=p['n_estimators'],
                max_depth=p['max_depth'],
                max_features=p['max_features'],
                min_samples_split=p['min_samples_split'],
                min_samples_leaf=p['min_samples_leaf'],
                bootstrap=p['bootstrap'],
                n_streams=4,
                random_state=42
            )
            rf.fit(X.iloc[tr_idx], y.iloc[tr_idx])
            pred = rf.predict(X.iloc[va_idx])
            r2_fold.append(r2_score(y.iloc[va_idx], pred))
            rmse_fold.append(np.sqrt(mean_squared_error(y.iloc[va_idx], pred)))

        r2_mean = float(np.mean(r2_fold))
        if r2_mean > best_r2 + min_delta:
            best_r2 = r2_mean
            best_params = p
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"⏹️ Early stopping after {no_improve} non-improving combos.")
                break

    elapsed = perf_counter() - start
    print(f"✅ Best CV R²={best_r2:.3f} | time={elapsed:.1f}s")
    return best_params, best_r2

def fit_best_cuml_model(X, y, best_params):
    rf = cuRF(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        max_features=best_params['max_features'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        bootstrap=best_params['bootstrap'],
        n_streams=4,
        random_state=42
    )
    rf.fit(X, y)
    return rf

# ----------------------------------------------------------
# 7. Train global model
# ----------------------------------------------------------
print("🔧 Tuning GLOBAL model (spatial CV + early stopping)...")
best_params_global, best_cv_r2_global = tune_with_spatial_cv_cuml(
    X, y, groups, grid_list, patience=12, min_delta=1e-3, desc_label="GLOBAL"
)
print("Best GLOBAL params:", best_params_global)

global_model = fit_best_cuml_model(X, y, best_params_global)
joblib.dump(global_model, os.path.join(model_dir, "model_GLOBAL.joblib"))

# ----------------------------------------------------------
# 8. Train per-country models
# ----------------------------------------------------------
country_models = {}
countries = df['country'].unique()

for c in countries:
    mask = (df['country'] == c)
    Xc, yc = X.loc[mask], y.loc[mask]
    groups_c = groups.loc[mask]

    print(f"\n🔧 Tuning {c} model (spatial CV + early stopping)...")
    best_params_c, best_cv_r2_c = tune_with_spatial_cv_cuml(
        Xc, yc, groups_c, grid_list, patience=10, min_delta=1e-3, desc_label=c
    )
    print(f"Best {c} params:", best_params_c)

    model_c = fit_best_cuml_model(Xc, yc, best_params_c)
    country_models[c] = {'model': model_c, 'params': best_params_c, 'cv_r2': best_cv_r2_c}
    joblib.dump(model_c, os.path.join(model_dir, f"model_{c}.joblib"))

# ----------------------------------------------------------
# 9. Evaluate all models across countries
# ----------------------------------------------------------
results = []
for train_country, model_info in [('GLOBAL', {'model': global_model})] + list(country_models.items()):
    model = model_info['model']
    for test_country in countries:
        mask = (df['country'] == test_country)
        X_test, y_test = X.loc[mask], y.loc[mask]
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        results.append({'train_model': train_country, 'test_country': test_country, 'r2': r2, 'rmse': rmse})

results_df = pd.DataFrame(results)
pivot_r2 = results_df.pivot(index='train_model', columns='test_country', values='r2')
pivot_rmse = results_df.pivot(index='train_model', columns='test_country', values='rmse')

print("\n✅ Model evaluation (R²):")
print(pivot_r2.round(3))
print("\n✅ Model evaluation (RMSE):")
print(pivot_rmse.round(1))

results_df.to_csv(os.path.join(model_dir, "spatialcv_results.csv"), index=False)
pivot_r2.to_csv(os.path.join(model_dir, "r2_matrix.csv"))
pivot_rmse.to_csv(os.path.join(model_dir, "rmse_matrix.csv"))

print(f"\nAll models & results saved in: {model_dir}")

🔹 Loading datasets...


/tmp/ipython-input-2991154303.py:27: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  yield_df = pd.read_stata(yield_path)


✅ Final merged shape: (4772, 91)
Total parameter combos (shuffled): 96
🔧 Tuning GLOBAL model (spatial CV + early stopping)...


GLOBAL:   0%|                                                                | 0/96 [00:00<?, ?it/s]

⏹️ Early stopping after 12 non-improving combos.
✅ Best CV R²=0.249 | time=143.9s
Best GLOBAL params: {'bootstrap': True, 'max_depth': 12, 'max_features': 0.8, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}

🔧 Tuning Ethiopia model (spatial CV + early stopping)...


...

⏹️ Early stopping after 10 non-improving combos.
✅ Best CV R²=0.223 | time=62.1s
Best Nigeria params: {'bootstrap': True, 'max_depth': 32, 'max_features': 0.5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}

🔧 Tuning Tanzania model (spatial CV + early stopping)...


Tanzania:   0%|                                                              | 0/96 [00:00<?, ?it/s]

⏹️ Early stopping after 10 non-improving combos.
✅ Best CV R²=0.145 | time=76.9s
Best Tanzania params: {'bootstrap': True, 'max_depth': 12, 'max_features': 0.8, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}

🔧 Tuning Uganda model (spatial CV + early stopping)...


Uganda:   0%|                                                                | 0/96 [00:00<?, ?it/s]

⏹️ Early stopping after 10 non-improving combos.
✅ Best CV R²=-0.018 | time=59.9s
Best Uganda params: {'bootstrap': True, 'max_depth': 32, 'max_features': 0.5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 300}

✅ Model evaluation (R²):
test_country  Ethiopia  Malawi   Mali  Nigeria  Tanzania  Uganda
train_model                                                     
Ethiopia         0.787  -0.188 -0.229   -1.605    -0.144  -0.029
GLOBAL           0.671   0.436  0.645    0.619     0.563   0.506
Malawi          -1.565   0.620 -1.803   -6.772    -0.048  -0.266
Mali            -2.930  -0.605  0.792   -2.034    -0.540  -1.097
Nigeria         -0.359  -1.162 -0.370    0.810    -1.117  -0.487
Tanzania        -0.281  -0.094 -0.031   -0.403     0.727  -0.437
Uganda          -0.591  -0.250 -1.259   -7.651    -0.624   0.752

✅ Model evaluation (RMSE):
test_country  Ethiopia  Malawi  Mali  Nigeria  Tanzania  Uganda
train_model                                                    
Ethio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
